In [ ]:
-- Reduce text output
SET NOCOUNT ON;

**Custom Table Value Function - Inline delimited Spliter.**  
[https://www.sqlservercentral.com/articles/tally-oh-an-improved-sql-8k-%E2%80%9Ccsv-splitter%E2%80%9D-function](https://www.sqlservercentral.com/articles/tally-oh-an-improved-sql-8k-%E2%80%9Ccsv-splitter%E2%80%9D-function)

In [ ]:
CREATE FUNCTION [dbo].[fn_DelimitedSplit8K]
--===== Define I/O parameters
        (@pString VARCHAR(8000), @pDelimiter CHAR(1))
-- WARNING!!! DONOT USE MAX DATA-TYPES HERE!  IT WILL KILL PERFORMANCE!
RETURNS TABLE WITH SCHEMABINDING AS
 RETURN
--===== "Inline" CTE Driven "Tally Table" produces values from 1 up to 10,000...
  WITH E1(N) AS (
                 SELECT 1 UNION ALL SELECT 1 UNION ALL SELECT 1 UNION ALL
                 SELECT 1 UNION ALL SELECT 1 UNION ALL SELECT 1 UNION ALL
                 SELECT 1 UNION ALL SELECT 1 UNION ALL SELECT 1 UNION ALL SELECT 1
                ),                          --10E+1 or 10 rows
       E2(N) AS (SELECT 1 FROM E1 a, E1 b), --10E+2 or 100 rows
       E4(N) AS (SELECT 1 FROM E2 a, E2 b), --10E+4 or 10,000 rows max
 cteTally(N) AS (--==== This provides the "base" CTE and limits the number of rows right up front
                     -- for both a performance gain and prevention of accidental "overruns"
                 SELECT TOP (ISNULL(DATALENGTH(@pString),0)) ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) FROM E4
                ),
cteStart(N1) AS (--==== This returns N+1 (starting position of each "element" just once for each delimiter)
                 SELECT 1 UNION ALL
                 SELECT t.N+1 FROM cteTally t WHERE SUBSTRING(@pString,t.N,1) = @pDelimiter
                ),
cteLen(N1,L1) AS(--==== Return start and length (for use in substring)
                 SELECT s.N1,
                        ISNULL(NULLIF(CHARINDEX(@pDelimiter,@pString,s.N1),0)-s.N1,8000)
                   FROM cteStart s
                )
--===== Do the actual split. The ISNULL/NULLIF combo handles the length for the final element when no delimiter is found.
 SELECT ItemNumber = ROW_NUMBER() OVER(ORDER BY l.N1),
        Item       = SUBSTRING(@pString, l.N1, l.L1)
   FROM cteLen l
;

In [4]:
-- Example using fn_DelimtedSplit
IF OBJECT_ID('tempdb..#Demo') IS NOT NULL DROP TABLE #Demo;
-- Add Values to demo
SELECT * INTO #Demo FROM (SELECT 1, 'This,is,a,demo.') as D (ID, CsvValue);
-- Split with cross apply
SELECT Demo.ID, Demo.CsvValue, split.ItemNumber, Item = split.Item
FROM #Demo as Demo CROSS APPLY dbo.fn_DelimitedSplit8k(Demo.CsvValue,',') split

Commands completed successfully.

Total execution time: 00:00:00.011

ID,CsvValue,ItemNumber,Item
1,"This,is,a,demo.",1,This
1,"This,is,a,demo.",2,is
1,"This,is,a,demo.",3,a
1,"This,is,a,demo.",4,demo.


**Custom Table Value Function -** **Split.**  
[https://qa.social.msdn.microsoft.com/Forums/sqlserver/en-US/0e536e2f-c3a1-471a-994e-50b8dea972f5/delimited-string-to-separate-rows?forum=transactsql](https://qa.social.msdn.microsoft.com/Forums/sqlserver/en-US/0e536e2f-c3a1-471a-994e-50b8dea972f5/delimited-string-to-separate-rows?forum=transactsql)

In [ ]:
CREATE FUNCTION [dbo].[fn_DoSplit](
    @sInputList VARCHAR(8000) -- List of delimited items
  , @sDelimiter VARCHAR(8000) = ',' -- delimiter that separates items
) RETURNS @List TABLE (item VARCHAR(8000))
BEGIN
DECLARE @sItem VARCHAR(8000)
WHILE CHARINDEX(@sDelimiter,@sInputList,0) <> 0
 BEGIN
 SELECT
  @sItem=RTRIM(LTRIM(SUBSTRING(@sInputList,1,CHARINDEX(@sDelimiter,@sInputList,0)-1))),
  @sInputList=RTRIM(LTRIM(SUBSTRING(@sInputList,CHARINDEX(@sDelimiter,@sInputList,0)+LEN(@sDelimiter),LEN(@sInputList))))

 IF LEN(@sItem) > 0
  INSERT INTO @List SELECT @sItem
 END

IF LEN(@sInputList) > 0
 INSERT INTO @List SELECT @sInputList -- Put the last item in
RETURN
END
GO

In [5]:
-- Example using fn_DoSplit
SELECT item from fn_DoSplit('One|Two|Three', '|')

Commands completed successfully.

Total execution time: 00:00:00.004

item
One
Two
Three


**Custom Scalar-valued Function -** Remove alpha chars from field

In [ ]:
CREATE FUNCTION [dbo].[fn_AlphaCharRemove](@Temp VARCHAR(1000))
RETURNS VARCHAR(1000)
AS
BEGIN
    DECLARE @KeepValues AS VARCHAR(50)
    SET @KeepValues = '%[^0-9]%'
    WHILE PATINDEX(@KeepValues, @Temp) > 0
        SET @Temp = STUFF(@Temp, PATINDEX(@KeepValues, @Temp), 1, '')
    RETURN @Temp
END
GO

In [14]:
-- Example using fn_AlphaCharRemove
Select [dbo].[fn_AlphaCharRemove]('1234?') as FixedNumber

Commands completed successfully.

Total execution time: 00:00:00.002

FixedNumber
1234


**Custom Scalar-valued Function -** Make text string proper case

[https://www.sqlservercentral.com/blogs/sql-server-tsql-to-convert-string-in-proper-format](https://www.sqlservercentral.com/blogs/sql-server-tsql-to-convert-string-in-proper-format)

In [ ]:
CREATE FUNCTION [dbo].[fn_Proper]
(
  @StrToConvertToProper AS VARCHAR(MAX)
) 
RETURNS VARCHAR(MAX) 
AS
BEGIN
  --Trim the Text
  SET @StrToConvertToProper = LTRIM(RTRIM(@StrToConvertToProper))
 
  --Find the No. of Words in the Text
  DECLARE @WordCount AS INT
  SET @WordCount = LEN(@StrToConvertToProper) - LEN(REPLACE(@StrToConvertToProper,' ','')) + 1
 
  --Variable to track the space position
  DECLARE @LastSpacePosition AS INT = 0
 
  --Loop through all the words
  WHILE(@WordCount > 0)
    BEGIN
      --Set the Space Position
      SET @LastSpacePosition = CHARINDEX(' ',@StrToConvertToProper,@LastSpacePosition + 1)
      
      --Replace the Character
      SET @StrToConvertToProper = STUFF(@StrToConvertToProper,
                                        @LastSpacePosition + 1,
                                        1,
                                        UPPER(SUBSTRING(@StrToConvertToProper, @LastSpacePosition + 1, 1)))
      --Decrement the Loop counter                                      
      SET @WordCount = @WordCount - 1
    END
    
  RETURN @StrToConvertToProper
END  

In [5]:
SELECT dbo.fn_Proper('hello, how are you?') as [Proper];

(1 row affected)

Total execution time: 00:00:00.003

Proper
"Hello, How Are You?"
